# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):

# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list[0])

/home/workspace
/home/workspace/event_data/2018-11-27-events.csv


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()


# To establish connection and begin executing queries, need a session
session = cluster.connect()


#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


# TASK 1: Getting the artist, song title, and song's length In the music app history 
#### For this query, sessionId is considered as partition key because it will give valuable information about per session
#### For this query, itemInsession is considered as cluster column because it will be used as sortting items number per session

In [8]:
#delete sessions table if exists
drop_query = "DROP TABLE IF EXISTS music_session"
session.execute(drop_query)

#to query by sessionId and itemInSession, i put sessionId as partition key and itemInSession as clustering columns to order by
query = "CREATE TABLE IF NOT EXISTS music_session"
query = query + "(sessionId INT, itemInSession INT, artist TEXT, length FLOAT, , song VARCHAR, PRIMARY KEY((sessionId), itemInSession));"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Open the csv file and INSERT the data to the table i created above

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_session (sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
#the reason where statement start with sessionId is because, Casssandra has to follow primary key order
select_session_query = """SELECT artist, song, length
                        FROM music_session
                        WHERE sessionId = 338 AND itemInSession = 4
                        """
rows = session.execute(select_session_query)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Task 2 : Getting the artist, song, and username(firstname, lastname)
#### For this query, userId and sessionId is considered as partition key because it will give song information specifically what userId = 10 listen in each session
#### For this query, itemInsession is considered as cluster column because in sessionId, it can be more devided to item in session number

In [14]:
#delete table users if exists
drop_query = "DROP TABLE IF EXISTS song_playlist_session"
session.execute(drop_query)

#userId and sessionId are for partition key and itemInSession is for cluster column which do order
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userId INT, sessionId INT, itemInSession INT, artist TEXT, firstName TEXT, lastName TEXT, song TEXT, PRIMARY KEY((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Open the csv file and INSERT the data to the table i created above

In [15]:
# insert data in to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
#userId and sessionId are used to find specific data, itemInSession is not written in where statement but used for order
select_user_query = """SELECT artist, firstName, lastName, song
                        FROM song_playlist_session
                        WHERE userId = 10 AND sessionId = 182
                        """
rows = session.execute(select_user_query)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Task 3 : Getting the user name who listen to the song 'All Hands Against His Own'
#### For this query, song is considered as partition key because the purpose of this query to find out username who listen to specific song 
#### For this query, user_id is considered as cluster column because there might be same name people

In [17]:
# delete usernames table if exists
drop_query = "DROP TABLE IF EXISTS user_listen"
session.execute(drop_query)

#song is the partition key and userId is cluster columns for order
query = "CREATE TABLE IF NOT EXISTS user_listen"
query = query + "(song TEXT, userId INT, firstName TEXT, lastName TEXT, PRIMARY KEY((song), userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Open the csv file and INSERT the data to the table i created above

In [18]:
# insert data in to table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listen (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

InvalidRequest: Error from server: code=2200 [Invalid query] message="Invalid STRING constant (Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)) for "userid" of type int"

#### Do a SELECT to verify that the data have been inserted into each table

In [22]:
#where song = '' is used to find specific data, and userId is for order
select_username_query = """SELECT firstName, lastName
                        FROM user_listen
                        WHERE song = 'All Hands Against His Own'
                        """
rows = session.execute(select_username_query)
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
drop_sessions_table = "DROP TABLE IF EXISTS music_session"
session.execute(drop_sessions_table)

drop_users_table = "DROP TABLE IF EXISTS song_playlist_session"
session.execute(drop_users_table)

drop_usernames_table = "DROP TABLE IF EXISTS user_listen"
session.execute(drop_usernames_table)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()